In [2]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor # Highly popular and efficient GBDT
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import SelectKBest, mutual_info_regression
import warnings

# Ignore minor warnings
warnings.filterwarnings('ignore')

print("="*70)
print("   SMURF HEART FAILURE PROJECT: PART 3 - TREE-BASED MODELS (DT, RF, XGB)")
print("="*70)

# Define the log-transform function (np.log1p) for y for variance stabilization
LOG_TRANSFORM = True 
K_FEATURES = 15 # Number of features to select

# ============================================================================
# STEP 1: LOAD DATA, PREPROCESSOR, AND FEATURE SELECTOR
# ============================================================================
print("\n[STEP 1] Loading data, preprocessor, and feature selector...")

# Load the preprocessor and feature selector from previous parts
preprocessor = joblib.load('smurf_preprocessor.joblib')
selector_mi = joblib.load('feature_selector.joblib')

# Load training data
df_X_train = pd.read_csv("X_train.csv")
df_y_train = pd.read_csv("y_train.csv", header=None, names=['heart_failure_risk'])
y_train = df_y_train['heart_failure_risk'].values

# Apply same column renaming
df_X_train_renamed = df_X_train.rename(columns={
    'smurfberry liquor': 'smurfberry_liquor',
    'smurfin donuts': 'smurfin_dots'
})

# Transform training data using fitted preprocessor
X_train_processed = preprocessor.transform(df_X_train_renamed)

# Select features using the loaded selector
X_train_selected = selector_mi.transform(X_train_processed)

# --- TARGET VARIABLE TRANSFORMATION ---
if LOG_TRANSFORM:
    y_train_transformed = np.log1p(y_train)
    print("Target variable y_train has been transformed using log(1+y).")
else:
    y_train_transformed = y_train

print(f"Selected training data shape: {X_train_selected.shape}")

# Get selected feature names (for plotting/reporting)
feature_names = (
    preprocessor.named_transformers_['num'].get_feature_names_out().tolist() +
    preprocessor.named_transformers_['ord'].get_feature_names_out().tolist() +
    preprocessor.named_transformers_['nom'].get_feature_names_out().tolist()
)
selected_features = [feature_names[i] for i in selector_mi.get_support(indices=True)]


# ============================================================================
# STEP 2: MODEL COMPARISON (TREE MODELS)
# ============================================================================
print("\n[STEP 2] Comparing Tree-based models...")
print("Using 5-fold cross-validation on training data...")

# Define models to compare
models_tree = {
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42, n_jobs=-1),
    'Gradient Boosting (SKL)': GradientBoostingRegressor(random_state=42),
    'XGBoost': XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1)
}

# Store results
cv_results = {}

print("\nRunning cross-validation for each model...")
print("-" * 70)

for name, model in models_tree.items():
    print(f"\nEvaluating {name}...")
    
    # Perform 5-fold cross-validation
    cv_scores = cross_val_score(
        model, X_train_selected, y_train_transformed,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=1
    )
    
    # Convert to RMSE
    rmse_scores = np.sqrt(-cv_scores)
    
    cv_results[name] = {
        'mean_rmse': rmse_scores.mean(),
        'std_rmse': rmse_scores.std(),
        'scores': rmse_scores
    }
    
    print(f"  Cross-Val RMSE: {rmse_scores.mean():.6f} (+/- {rmse_scores.std():.6f})")

# Display results summary
print("\n" + "="*70)
print("CROSS-VALIDATION RESULTS SUMMARY (on Transformed Training Data)")
print("="*70)
results_df = pd.DataFrame({
    'Model': list(cv_results.keys()),
    'Mean CV RMSE': [cv_results[m]['mean_rmse'] for m in cv_results.keys()],
    'Std CV RMSE': [cv_results[m]['std_rmse'] for m in cv_results.keys()]
}).sort_values('Mean CV RMSE')

print(results_df.to_string(index=False))

# ============================================================================
# STEP 3: HYPERPARAMETER TUNING FOR TOP MODELS
# ============================================================================
print("\n[STEP 3] Hyperparameter tuning for top 3 models...")

# Select top 3 models
top_3_models = results_df.head(3)['Model'].tolist()
print(f"\nTuning hyperparameters for: {top_3_models}")

tuned_models = {}

# Define hyperparameter grids
param_grids_tree = {
    'Random Forest': {
        'n_estimators': [100, 300, 500],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'Gradient Boosting (SKL)': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 0.9, 1.0]
    },
    'XGBoost': {
        'n_estimators': [100, 300, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.9, 1.0]
    },
    'Decision Tree': {
        'max_depth': [5, 10, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 5, 10]
    }
}

for model_name in top_3_models:
    print(f"\n  Tuning {model_name}...")
    
    # Use RandomizedSearchCV for faster tuning
    model_to_tune = models_tree[model_name]

    search = RandomizedSearchCV(
        model_to_tune,
        param_grids_tree[model_name],
        n_iter=20, # Try 20 random combinations
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1, # Use all cores
        random_state=42,
        verbose=0
    )
    
    search.fit(X_train_selected, y_train_transformed)
    
    tuned_models[model_name] = {
        'model': search.best_estimator_,
        'best_params': search.best_params_,
        'best_rmse': np.sqrt(-search.best_score_)
    }
    
    print(f"    Best CV RMSE: {tuned_models[model_name]['best_rmse']:.6f}")
    print(f"    Best parameters: {search.best_params_}")

# ============================================================================
# STEP 4: SELECT BEST MODEL AND TRAIN ON FULL TRAINING SET
# ============================================================================
print("\n[STEP 4] Selecting best model...")

# Find the best tuned model
best_model_name = min(tuned_models.keys(), 
                      key=lambda x: tuned_models[x]['best_rmse'])
best_model = tuned_models[best_model_name]['model']

print(f"\n*** BEST TREE-BASED MODEL: {best_model_name} ***")
print(f"CV RMSE (Transformed): {tuned_models[best_model_name]['best_rmse']:.6f}")

# Train on full training set
print("\nTraining best model on full training set (transformed y)...")
best_model.fit(X_train_selected, y_train_transformed)

# Evaluate on training set
y_train_pred_transformed = best_model.predict(X_train_selected)
# --- INVERSE TRANSFORM FOR METRICS ---
y_train_pred = np.expm1(y_train_pred_transformed) 

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)

print(f"\nTraining Set Performance (Original Scale):")
print(f"  RMSE: {train_rmse:.6f}")
print(f"  R²:   {train_r2:.6f}")
print(f"  MAE:  {train_mae:.6f}")

# ============================================================================
# STEP 5: EVALUATE ON TEST SET (FINAL EVALUATION)
# ============================================================================
print("\n[STEP 5] Final evaluation on test set...")

# Load test data
df_X_test = pd.read_csv("X_test.csv")
df_y_test = pd.read_csv("y_test.csv", header=None, 
                        names=['heart_failure_risk'])
y_test = df_y_test['heart_failure_risk'].values

# Apply same preprocessing
df_X_test_renamed = df_X_test.rename(columns={
    'smurfberry liquor': 'smurfberry_liquor',
    'smurfin donuts': 'smurfin_dots'
})

X_test_processed = preprocessor.transform(df_X_test_renamed)
X_test_selected = selector_mi.transform(X_test_processed)

# Make predictions on the test set
y_test_pred_transformed = best_model.predict(X_test_selected)
# --- INVERSE TRANSFORM PREDICTIONS ---
y_test_pred = np.expm1(y_test_pred_transformed)

# Calculate metrics (using original y_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print("\n" + "="*70)
print("             FINAL TEST SET PERFORMANCE (Original Scale)")
print("="*70)
print(f"Model: {best_model_name}")
print(f"\nTest Set Metrics:")
print(f"  RMSE: {test_rmse:.6f}")
print(f"  R²:   {test_r2:.6f}")
print(f"  MAE:  {test_mae:.6f}")
print("="*70)

# ============================================================================
# STEP 6: SAVE MODEL AND RESULTS
# ============================================================================
print("\n[STEP 6] Saving the best model and results...")

# Save the best model
joblib.dump(best_model, 'best_tree_model.joblib')

# Save comprehensive results
results_summary = {
    'best_model_name': best_model_name,
    'cv_rmse_transformed': tuned_models[best_model_name]['best_rmse'],
    'test_rmse': test_rmse,
    'test_r2': test_r2,
    'test_mae': test_mae,
    'tuned_models': {k: {'params': v['best_params'], 'rmse': v['best_rmse']} for k, v in tuned_models.items()}
}
joblib.dump(results_summary, 'part3_results_summary.joblib')

print("\nSaved files:")
print("  - best_tree_model.joblib")
print("  - part3_results_summary.joblib")

# ============================================================================
# STEP 7: GENERATE VISUALIZATIONS
# ============================================================================
print("\n[STEP 7] Generating visualizations...")

# 1. Model comparison
plt.figure(figsize=(10, 6))
plt.barh(results_df['Model'], results_df['Mean CV RMSE'], xerr=results_df['Std CV RMSE'], capsize=5)
plt.xlabel('Cross-Validation RMSE (Transformed y)')
plt.title('Tree Model Comparison (5-Fold Cross-Validation)')
plt.tight_layout()
plt.savefig('tree_model_comparison.png', dpi=300, bbox_inches='tight')
print("  Saved: tree_model_comparison.png")

# 2. Residuals plot (Crucial check for Heteroscedasticity)
residuals = y_test - y_test_pred # Residuals on ORIGINAL scale
plt.figure(figsize=(10, 6))
plt.scatter(y_test_pred, residuals, alpha=0.5, edgecolors='k', linewidths=0.5)
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel('Predicted Values (Original Scale)')
plt.ylabel('Residuals (Original Scale)')
plt.title(f'Residual Plot ({best_model_name}) - Test Set')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('residuals_plot_tree.png', dpi=300, bbox_inches='tight')
print("  Saved: residuals_plot_tree.png")

print("\n" + "="*70)
print("PART 3 COMPLETE!")
print("="*70)

   SMURF HEART FAILURE PROJECT: PART 3 - TREE-BASED MODELS (DT, RF, XGB)

[STEP 1] Loading data, preprocessor, and feature selector...
Target variable y_train has been transformed using log(1+y).
Selected training data shape: (1000, 15)

[STEP 2] Comparing Tree-based models...
Using 5-fold cross-validation on training data...

Running cross-validation for each model...
----------------------------------------------------------------------

Evaluating Decision Tree...
  Cross-Val RMSE: 0.052686 (+/- 0.002319)

Evaluating Random Forest...
  Cross-Val RMSE: 0.039560 (+/- 0.002490)

Evaluating Gradient Boosting (SKL)...
  Cross-Val RMSE: 0.037997 (+/- 0.003005)

Evaluating XGBoost...
  Cross-Val RMSE: 0.041265 (+/- 0.002344)

CROSS-VALIDATION RESULTS SUMMARY (on Transformed Training Data)
                  Model  Mean CV RMSE  Std CV RMSE
Gradient Boosting (SKL)      0.037997     0.003005
          Random Forest      0.039560     0.002490
                XGBoost      0.041265     0.002344


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.
